# TensorBoard Integration with LatticeMC

This notebook demonstrates how to use TensorBoard to visualize simulation progress with immediate logging.

In [ ]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.WARNING)
# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# add formatter to console handler
ch.setFormatter(formatter)
# add console handler to root logger
root.addHandler(ch)

In [ ]:
import numpy as np
from decimal import Decimal

In [ ]:
from latticemc.definitions import Lattice, LatticeState, OrderParametersHistory, DefiningParameters
from latticemc.lattice_tools import initialize_partially_ordered
from latticemc.random_quaternion import random_quaternion
from latticemc.updaters import AcceptanceRateWiggleRateAdjustor, RandomWiggleRateAdjustor
from latticemc.parallel import SimulationRunner

## Set up simulation with TensorBoard logging enabled

We'll create a simulation with multiple temperatures and enable TensorBoard logging.
With the new immediate logging approach, data is logged as soon as it's received from simulation processes.

In [ ]:
# Setup simulation with multiple temperatures
temperatures = np.arange(0.3, 1.7, 0.05)  # Fewer temps for faster demo
states = [LatticeState(parameters=DefiningParameters(temperature=round(Decimal(t), 2), lam=Decimal(0.3), tau=1),
                       lattice=Lattice(8, 8, 8))  # Smaller lattice for faster demo
          for t in temperatures]
for state in states:
    initialize_partially_ordered(state.lattice, x=random_quaternion(1.0))

order_parameters_history = {state.parameters: OrderParametersHistory() for state in states}

In [ ]:
# Setup updaters
per_state_updaters = [
    AcceptanceRateWiggleRateAdjustor(how_often=50, since_when=10),
    RandomWiggleRateAdjustor(scale=0.001, how_often=10, since_when=10),
    RandomWiggleRateAdjustor(scale=1.0, reset_value=1.0, how_often=500, since_when=500)
]

# Create simulation runner with TensorBoard logging enabled
runner = SimulationRunner(states,
                         order_parameters_history,
                         cycles=10000,  # Fewer cycles for faster demo
                         report_order_parameters_every=100,
                         report_state_every=500,
                         per_state_updaters=per_state_updaters,
                         parallel_tempering_interval=50,
                         tensorboard_log_dir='./tensorboard_logs')

## Start the simulation

Run the simulation and TensorBoard logging will happen automatically as data is received:
- Order parameters are logged immediately when received
- Fluctuations are logged immediately when received  
- Lattice states (including energy and acceptance rates) are logged when state updates are received

In [ ]:
# Start the simulation
runner.start()

## Launch TensorBoard

While the simulation is running, you can launch TensorBoard in a new terminal to visualize the progress:

```bash
tensorboard --logdir=./tensorboard_logs
```

This will start a local server, typically at http://localhost:6006, where you can view the visualizations.

### What you'll see in TensorBoard:

1. **Order Parameters**: Real-time plots of energy, q0, q2, w, p, d322 for each temperature
2. **Fluctuations**: Real-time fluctuation measurements for each order parameter
3. **Acceptance Rates**: Live monitoring of orientation and parity move acceptance rates
4. **Lattice Energy**: Energy values from lattice averages updated as states are received

In [ ]:
# If you want to stop the simulation early
# runner.stop()

In [ ]:
# Check if the simulation is still running
runner.alive()

In [ ]:
# When done, check if it finished properly
runner.finished_gracefully()

## Benefits of Immediate Logging

The new immediate logging approach provides several advantages:

1. **Real-time monitoring**: See data as soon as it's computed
2. **Lower memory usage**: No need to batch data before logging
3. **Better responsiveness**: TensorBoard updates continuously during simulation
4. **Specific data logging**: Only log what's actually received/computed
5. **Failure resilience**: Data is logged even if simulation crashes partway through